## Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio import features
from autoRIFT import autoRIFT
from collections import namedtuple
from pathlib import Path
import itertools
import scipy.io as sio
import sys
import shutil
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")
# from autoRIFT import autoriftcore
# from geogrid import GeogridOptical

In [2]:
pwd

'/home/micromamba/scripts'

In [ ]:
autorift_cmd = str(Path('testautoRIFT.py').resolve())
geogrid_cmd = str(Path('testGeogridOptical.py').resolve())

In [3]:
cd /home/micromamba/data/spot/landslide_54

/home/micromamba/data/spot


## Functions

In [ ]:
def read_raster(path,bands=1,crs=False,bounds=False):
    # From rasterio docs with modifications
    with rio.open(path) as dst:
        array = dst.read(bands)
        profile = dst.profile
        crs_val = dst.crs
        bounds_val = dst.bounds
        # array = np.moveaxis(array,0,-1)
    
    result = [array, profile]

    if crs:
        result.append(crs_val)
    
    if bounds:
        result.append(bounds_val)

    return result


def write_raster(array,profile,out_path,nodata,dtype,n_bands=1):
    # From rasterio docs:
    # Register GDAL format drivers and configuration options with a
    # context manager.
    with rio.Env():
        # And then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile.update(
            dtype=dtype,
            count=n_bands,
            nodata=nodata,
            compress='lzw')

        with rio.open(out_path, 'w', **profile) as dst:
            if n_bands == 1:
                dst.write(array.astype(dtype),1)
            else:
                dst.write(array.astype(dtype))

    return out_path

def polygon_to_raster(gdf,template_path,value=1,crs=False):
    if isinstance(gdf,str):
        pol = gpd.read_file(gdf)
    else:
        pol = gdf

    with rio.open(template_path) as dst:
        profile = dst.profile
        template_crs = dst.crs
        template_transform = profile['transform']
        template_shape = dst.shape

    # if crs != pol.crs:
    #   raise Exception('CRSs do not match!')

    geojsons = [x['geometry'] for x in pol.geometry.__geo_interface__['features']]
    if isinstance(value,str):
        shapes = [tuple(x) for x in zip(geojsons,pol[value])]
    else:
        shapes = [(x,value) for x in geojsons]

    array = features.rasterize(shapes, out_shape=template_shape, transform=template_transform)
    
    result = [array, profile]
    if crs:
        result.append(template_crs)

    return result

## AutoRIFT Inputs
**I1**  reference image (extracted image patches defined as "source")

**I2**  secondary image (extracted image patches defined as "template"; displacement = motion vector of I2 relative to I1 which should be acquired earlier in our convention)

**xGrid** [units = integer image pixels]  horizontal reference image pixel index at each grid point

**yGrid**  [units = integer image pixels]  vertical reference image pixel index at each grid point
(if xGrid and yGrid not provided, a regular grid spanning the entire image will be automatically set up, which is similar to the conventional ISCE module, "ampcor" or "denseampcor")

**Dx0**  [units = integer image pixels]  horizontal "downstream" search location (that specifies the horizontal pixel displacement of the template's search center relative to the source's) at each grid point

**Dy0**  [units = integer image pixels]  vertical "downstream" reach location (that specifies the vertical pixel displacement of the template's search center relative to the source's) at each grid point
(if Dx0 and Dy0 not provided, an array with zero values will be automatically assigned and there will be no offsets of the search centers)

**ChipSizeMinX** [units = integer image pixels]  Minimum size (in horizontal direction) of the template (chip) to correlate (default = 32; could be scalar or array with same dimension as xGrid)

**ChipSizeMaxX** [units = integer image pixels]  Maximum size (in horizontal direction) of the template (chip) to correlate (default = 64; could be scalar or array with same dimension as xGrid)

**ChipSize0X** [units = integer image pixels]  Minimum acceptable size (in horizontal direction) of the template (chip) to correlate (default = 32)

**GridSpacingX** [units = integer image pixels]  Grid Spacing (in horizontal direction) (default = 32; note GridSpacingX can be smaller than ChipSize0X leading to dependent chips)

**ScaleChipSizeY** [unitless; integer data type]  Scaling factor to get the vertical chip size in reference to the horizontal size (default = 1)

**SearchLimitX** [units = integer image pixels]  Range or limit (in horizontal direction) to search for displacement in the source (default = 25; could be scalar or array with same dimension as xGrid; when provided in array, set its elements to 0 if no search is desired in certain areas)

**SearchLimitY** [units = integer image pixels]  Range or limit (in vertical direction) to search for displacement in the source (default = 25; could be scalar or array with same dimension as xGrid; when provided in array, set its elements to 0 if no search is desired in certain areas)

**SkipSampleX** [units = integer image pixels] Number of samples to skip between search windows in horizontal direction if no grid specified by the user (default = 32)

**SkipSampleY** [units = integer image pixels]  Number of lines to skip between search windows in vertical direction if no grid specified by the user (default = 32)

**minSearch** [units = integer image pixels]  Minimum search range/limit (default = 6)

## Images Prep

In [ ]:
# Image = namedtuple('Image','date file')

types = {'sentinel2':{'files':'S2*tif','date_loc':slice(11,19)}}
desired_epsg = 32760

image_loc = Path('raw')
meta = types['sentinel2']
pattern = meta['files']
date_loc = meta['date_loc']

In [ ]:
images = {x.name[date_loc]:x for x in image_loc.glob(pattern)}
images

## Run Autorift

In [ ]:
pairs = []
for dates in itertools.combinations(images,2):
    
    dates = [datetime.strptime(x,'%Y%m%d') for x in dates]
    dates.sort()
    dates = [datetime.strftime(x,'%Y%m%d') for x in dates]
    
    pairs.append((images[dates[0]],images[dates[1]]))
    print(dates)
print(len(pairs))

In [ ]:
out_dir = Path('autorift')
if out_dir.exists():
    shutil.rmtree(out_dir)

out_dir.mkdir()

In [ ]:
ref, sec = [str(x) for x in pairs[0]]
base_dem = '../dem.tif'
dem = 'dem_window.tif'
res = 80

In [ ]:
!rm dem_window.tif

In [ ]:
_, _, crs, bbox = read_raster(ref,crs=True,bounds=True)
bbox = ' '.join([str(x) for x in bbox])
epsg = f'EPSG:{crs.to_epsg()}'

In [ ]:
!gdalwarp -t_srs {epsg} -te {bbox} -tr {res} {res} -te_srs {epsg} -r bilinear {base_dem} {dem}

In [ ]:
# arr, profile = read_raster(dem)

# def create_param(val,path,profile):
#     # data = np.zeros((10,10,2))
#     data = np.zeros((profile['height']-1,profile['width']-1,2))
    
#     data[:,:,0] = val
#     data[:,:,1] = val
#     data = np.moveaxis(data, [0, 1, 2], [2, 1, 0])
    
#     profile['count'], profile['width'], profile['height'] = data.shape 
#     write_raster(data,profile,path,nodata=0,dtype=rio.int8,n_bands=2)

#     return path

# csmin = create_param(32,'csmin.tif',profile)
# csmax = create_param(64,'csmax.tif',profile)
# sr = create_param(5,'sr.tif',profile)

In [ ]:
! {geogrid_cmd} -m {ref} -s {sec} -d {dem}

## Check

In [ ]:
%%capture --no-stderr

for i, (ref, sec) in enumerate(pairs):
    ! {autorift_cmd} -m {str(ref)} -s {str(sec)} -g window_location.tif -fo 1

    offset, offset_profile = read_raster('offset.tif',bands=[1,2,3,4])

    base_name = f'autorift_{ref.name[date_loc]}T000000_{sec.name[date_loc]}T000000'
    dx_name = out_dir / (base_name + '_EWOffset.tif')
    dy_name = out_dir / (base_name + '_NSOffset.tif')
    
    write_raster(offset[0,:,:]*10,offset_profile,dx_name,nodata=-32768,dtype=rio.float32)
    write_raster(offset[1,:,:]*10,offset_profile,dy_name,nodata=-32768,dtype=rio.float32)

In [ ]:
!rm offset.mat offset.tif window_location.tif

## Remove Median

In [ ]:
corrected_dir = Path('corrected')

In [ ]:
if corrected_dir.exists():
    shutil.rmtree(corrected_dir)

corrected_dir.mkdir()

ls = gpd.read_file('/home/micromamba/data/landslides/landslide_sediment.shp').to_crs(epsg)

In [ ]:
autorift_results = list(out_dir.glob('*Offset.tif'))
mask, _ = polygon_to_raster(ls.buffer(500),autorift_results[0],value=1,crs=False)

In [ ]:
plt.imshow(mask)

In [ ]:
all_vals = []

for ras in autorift_results:
    data, profile = read_raster(ras)
    
    tmp = data.copy()
    tmp[mask == 1] = np.nan
    median = np.nanmedian(tmp)
    data = data - median
    write_raster(data,profile,corrected_dir / ras.name,nodata=-32768,dtype=rio.float32)
    
    all_vals.append(median)

print(f'Offset image median min: {np.min(all_vals):.2f} mean: {np.mean(all_vals):.2f} max: {np.max(all_vals):.2f}')

## Create MintPy metadata file

In [ ]:
files = [x.name for x in corrected_dir.glob('*Offset.tif')]
files.sort()

lines = [f'{f} {f[9:17]} {f[25:33]}\n' for f in files]
with open(corrected_dir / 'metadata.txt', 'w') as f:
    f.writelines(lines)

## Notify

In [ ]:
from IPython.lib.display import Audio
import numpy as np

framerate = 4410
play_time_seconds = 2

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)